In [64]:
import pandas as pd
import seaborn as sns
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import os
import sys
from ocp_table_tpot.globals import Globals as gd
from tpot import TPOTRegressor
sys.path.insert(0,'..')
from src.models.model import HistoricalMedian,XGBoost,LinearModel,RF,KNN,SVM,mase,TimeSeriesSplitImproved
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from skgarden.quantile import RandomForestQuantileRegressor
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.preprocessing import FunctionTransformer
from copy import copy
from tpot.builtins import StackingEstimator
from lightgbm import LGBMRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import make_pipeline, make_union
from catboost import CatBoostRegressor,Pool,cv


df_tsfresh = pd.read_pickle(f'../data/processed/train_test_tsfresh_6.pkl').reset_index(level = 0)
data_dict = pd.read_pickle(f'../data/processed/data_dict_all.pkl')


,level_0,value__kurtosis_p6,"value__linear_trend__attr_""intercept""_p6","value__linear_trend__attr_""slope""_p6",value__quantile__q_0.01_p6,value__quantile__q_0.99_p6,value__skewness_p6
id,,,,,,,
1,rougher.input.feed_fe,-0.284606,8.290417,0.000000,6.100378,6.100378,0.000000
2,rougher.input.feed_fe,-0.284606,6.100378,0.060735,6.100985,6.160506,0.000000
3,rougher.input.feed_fe,-0.284606,6.117944,0.008038,6.100700,6.160220,1.237514
4,rougher.input.feed_fe,1.047950,6.137694,-0.021587,6.045021,6.159774,-0.366498
5,rougher.input.feed_fe,-0.780750,6.135780,-0.019673,6.044013,6.159327,0.345700


In [71]:
root='..'
df_flats = []
for p in [36,6,24]:
    df_tsfresh = pd.read_pickle(f'{root}/data/processed/train_test_tsfresh_{p}.pkl').reset_index(level=0)
    tmp =df_tsfresh.pivot_table(index='id', columns=['level_0'])
    tmp.columns = ['_'.join(col).strip() for col in tmp.columns.values]
    df_flats.append(tmp)

df_flat = pd.concat(df_flats,axis=1)

df_flat.index.names = ['date']
df_flat.head()
tmp

,value__kurtosis_p24_primary_cleaner.input.copper_sulfate,value__kurtosis_p24_primary_cleaner.input.depressant,value__kurtosis_p24_primary_cleaner.input.feed_size,value__kurtosis_p24_primary_cleaner.input.xanthate,value__kurtosis_p24_primary_cleaner.state.floatbank8_a_air,value__kurtosis_p24_rougher.input.feed_fe,value__kurtosis_p24_rougher.input.feed_pb,value__kurtosis_p24_rougher.input.feed_rate,value__kurtosis_p24_rougher.input.feed_sol,value__kurtosis_p24_rougher.input.feed_zn,...,value__skewness_p24_primary_cleaner.state.floatbank8_a_air,value__skewness_p24_rougher.input.feed_fe,value__skewness_p24_rougher.input.feed_pb,value__skewness_p24_rougher.input.feed_rate,value__skewness_p24_rougher.input.feed_sol,value__skewness_p24_rougher.input.feed_zn,value__skewness_p24_rougher.input.floatbank10_copper_sulfate,value__skewness_p24_rougher.input.floatbank11_xanthate,value__skewness_p24_rougher.state.floatbank10_b_air,value__skewness_p24_secondary_cleaner.state.floatbank5_a_air
id,,,,,,,,,,,,,,,,,,,,,
1,-0.278897,0.053372,-0.176947,-0.154139,-0.214268,0.000000,-0.100395,0.641409,0.719380,0.000000,...,-0.058141,-0.263996,-0.259988,-0.860201,-0.837851,-0.323986,-0.252958,-0.368861,-0.033020,-0.038588
2,-0.278897,0.053372,-0.176947,-0.154139,-0.214268,0.000000,-0.100395,0.641409,0.719380,0.000000,...,-0.058141,-0.263996,-0.259988,-0.860201,-0.837851,-0.323986,-0.252958,-0.368861,-0.033020,-0.038588
3,-0.278897,0.053372,-0.176947,-0.154139,-0.214268,0.000000,-0.100395,0.641409,0.719380,0.000000,...,-0.058006,1.237514,-1.581082,1.219928,1.437744,-1.708960,-1.351549,1.725034,0.664107,-1.403274
4,-1.742939,-5.661114,0.000000,-1.441736,-0.975720,1.047950,-1.018959,1.847391,-5.236670,1.551123,...,-0.909072,-0.366498,-0.868829,1.447526,-0.118080,-1.423029,-1.836277,0.001344,1.243404,-0.188030
5,0.666613,-2.156010,0.000000,-1.078796,0.531150,-0.780750,0.735865,-2.847330,0.963174,1.352728,...,-1.295466,0.345700,-1.141334,0.333572,-1.059807,-1.380184,-1.708991,-0.605228,1.521760,0.430250
6,-1.234524,-1.249255,0.000000,1.824896,1.552501,3.965021,-1.741339,-2.477429,0.297311,-2.136735,...,-1.552102,-1.867800,-0.379789,-0.038194,-0.652324,-0.519982,-0.706237,-0.964531,1.260495,-0.130560
7,-1.888178,-1.451355,0.000000,4.347661,2.521608,4.559010,-1.019515,-2.243768,-0.886157,-1.669840,...,-1.766990,-1.953136,-0.499886,0.311307,-0.153746,-0.365018,-0.643948,-1.226172,1.283002,-0.189063
8,-1.642567,-1.293400,0.000000,-0.456864,3.386386,2.338624,-0.865392,-2.073712,-0.476416,-1.341732,...,-1.955110,-1.446646,-0.644398,0.111052,-0.346535,-0.631005,0.001970,2.109900,0.940207,-0.244638
9,-0.554306,-1.001338,0.000000,-1.668334,4.138874,-0.144432,-0.687321,-1.068359,-0.066269,-1.008853,...,-2.118285,-0.874414,-0.616591,0.333733,-0.344318,-0.791393,0.598911,1.285839,1.133190,-0.482942


In [62]:
year = 2016
tgt = 'final.output.recovery'
X = data_dict[year]['X_train_ts']
y = data_dict[year]['y_train']


,primary_cleaner.input.copper_sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,...,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___p_r_i_m_a_r_y___c_l_e_a_n_e_r_._s_t_a_t_e_._f_l_o_a_t_b_a_n_k_8___a___a_i_r,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_e_e_d___f_e,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_e_e_d___p_b,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_e_e_d___r_a_t_e,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_e_e_d___s_o_l,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_e_e_d___z_n,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_l_o_a_t_b_a_n_k_1_0___c_o_p_p_e_r___s_u_l_f_a_t_e,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_l_o_a_t_b_a_n_k_1_1___x_a_n_t_h_a_t_e,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._s_t_a_t_e_._f_l_o_a_t_b_a_n_k_1_0___b___a_i_r,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___s_e_c_o_n_d_a_r_y___c_l_e_a_n_e_r_._s_t_a_t_e_._f_l_o_a_t_b_a_n_k_5___a___a_i_r
date,,,,,,,,,,,,,,,,,,,,,


In [60]:
X_test=  data_dict[year]['X_test']

print(f'1) Test shape: {X_test.shape}, train: {X.shape}')
    
inds_y = y[(y[tgt] > 5) & (y[tgt] < 100)].index
inds_common = inds_y

X = X.loc[inds_common,]
y = y.loc[inds_common, tgt]

X = X.sample(frac=0.3,random_state=123).sort_index().dropna()
y= y[X.index]



Nmonths_total = 8
Nspl = int(Nmonths_total * 30 / 15)
Nmonths_test = 4
Nmonths_min_train = 2.5
train_splits = Nspl // Nmonths_total*Nmonths_min_train
test_splits=int(Nmonths_test / Nmonths_total*Nspl)
cv = TimeSeriesSplitImproved(n_splits=Nspl)

1) Test shape: (2928, 53), train: (0, 305)


KeyError: "None of [DatetimeIndex(['2016-01-15 00:00:00+00:00', '2016-01-15 01:00:00+00:00',\n               '2016-01-15 02:00:00+00:00', '2016-01-15 03:00:00+00:00',\n               '2016-01-15 04:00:00+00:00', '2016-01-15 05:00:00+00:00',\n               '2016-01-15 06:00:00+00:00', '2016-01-15 07:00:00+00:00',\n               '2016-01-15 08:00:00+00:00', '2016-01-15 09:00:00+00:00',\n               ...\n               '2016-08-31 14:59:59+00:00', '2016-08-31 15:59:59+00:00',\n               '2016-08-31 16:59:59+00:00', '2016-08-31 17:59:59+00:00',\n               '2016-08-31 18:59:59+00:00', '2016-08-31 19:59:59+00:00',\n               '2016-08-31 20:59:59+00:00', '2016-08-31 21:59:59+00:00',\n               '2016-08-31 22:59:59+00:00', '2016-08-31 23:59:59+00:00'],\n              dtype='datetime64[ns, UTC]', name='date', length=4902, freq=None)] are in the [index]"

,primary_cleaner.input.copper_sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,...,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___p_r_i_m_a_r_y___c_l_e_a_n_e_r_._s_t_a_t_e_._f_l_o_a_t_b_a_n_k_8___a___a_i_r,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_e_e_d___f_e,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_e_e_d___p_b,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_e_e_d___r_a_t_e,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_e_e_d___s_o_l,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_e_e_d___z_n,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_l_o_a_t_b_a_n_k_1_0___c_o_p_p_e_r___s_u_l_f_a_t_e,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._i_n_p_u_t_._f_l_o_a_t_b_a_n_k_1_1___x_a_n_t_h_a_t_e,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___r_o_u_g_h_e_r_._s_t_a_t_e_._f_l_o_a_t_b_a_n_k_1_0___b___a_i_r,v_a_l_u_e_____s_k_e_w_n_e_s_s___p_2_4___s_e_c_o_n_d_a_r_y___c_l_e_a_n_e_r_._s_t_a_t_e_._f_l_o_a_t_b_a_n_k_5___a___a_i_r
date,,,,,,,,,,,,,,,,,,,,,


# Base models

In [22]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.005, random_state=1))

ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.005, l1_ratio=.9, random_state=3))

CatBoost = CatBoostRegressor(loss_function='MAE',random_seed =123,learning_rate=0.1,max_depth=8,task_type='GPU',od_type = 'Iter',od_wait= 15,iterations = 2000)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.15, max_depth=12, 
                             n_estimators=500,
                             reg_alpha=0.4, reg_lambda=0.8,
                             subsample=0.5, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='mae',num_leaves=5,
                              learning_rate=0.15, n_estimators=500,
                              bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9  )
param_grids = {'n_estimators': 1000,
                   'max_features': 0.8, # tuned
                   'max_depth': 14, # tuned
                   }
model_qrf = RandomForestQuantileRegressor(**param_grids,
               criterion = 'mae',
               n_jobs = -1,
                random_state =123)

In [23]:
n_folds = 5

def rmsle_cv(model,cv = None):
    if cv is None:
        cv = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X.values)
    score  = make_scorer(mase,greater_is_better=False)
    #rmse= np.sqrt(-cross_val_score(model, X.values, y.values.reshape(-1,1), scoring=score, cv = cv))
    rmse= cross_val_score(model, X.values, y.values.reshape(-1,), scoring=score, cv = cv)
    
    return(rmse)

def rmsle_cv_boxcox_tgt(model,cv = None,lam = 4.321):
    from scipy.stats import boxcox
    from scipy.special import inv_boxcox
    if cv is None:
        cv = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X.values)
    scores = []
    for fold_n, (train_index, valid_index) in enumerate(cv.split(X,fixed_length=False, train_splits=train_splits, test_splits=test_splits)):
    # print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        y_train = boxcox(y_train,lam)
        model.fit(X_train,y_train)
        preds  = inv_boxcox(model.predict(X_valid),4.321)
        score_val = mase(preds,y_valid)
        # print(f'Fold {fold_n}. Score: {score_val:.4f}.')
        print('')
        scores.append(score_val)
    
    #print(f'CV mean score: {np.mean(scores):.4f}, std: {np.std(scores):.4f}.')
    
    return(np.array(scores))


class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [107]:
score = rmsle_cv(lasso,cv = cv)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3


Lasso score: -3.1228 (0.3227)



C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3


Lasso score: -2.9564 (0.4491)



In [108]:
score = rmsle_cv(ENet,cv=cv)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(ENet,cv=None)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3

ElasticNet score: -3.1416 (0.3207)



C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3

ElasticNet score: -2.9653 (0.4515)



In [98]:
score = rmsle_cv(GBoost,cv=cv)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

KeyboardInterrupt: 

In [109]:
score = rmsle_cv(model_xgb,cv=cv)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Xgboost score: -2.5367 (0.2999)

Xgboost score: -2.1219 (0.1754)



In [110]:
score = rmsle_cv(model_lgb,cv=cv)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

LGBM score: -3.5367 (0.2241)

LGBM score: -3.0009 (0.2294)



In [104]:
score = rmsle_cv(model_qrf,cv=cv)
print("QRFscore: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
score = rmsle_cv(model_qrf)
print("QRF score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

LGBM score: -3.4970 (0.6479)



KeyboardInterrupt: 

In [19]:
pool = Pool(X, y)
params = {'iterations': 3000, 
          'loss_function': 'MAE', 
          'verbose': False,
         "random_seed":123,
          "learning_rate":0.1,
          "max_depth":8,
          "task_type":'GPU',
          "od_type" : 'Iter',
          "od_wait": 15
          }
scores = cv(pool, params)
#print("Catboostmodels score: {:.4f} ({:.4f})".format(scores.mean(), scores.std()))
scores.tail()

KeyboardInterrupt: 

In [122]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, lasso,model_xgb),
                                                 meta_model = lasso)
score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3\envs\ocp\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abiryukov\AppData\Local\Continuum\anaconda3

Stacking Averaged models score: -3.0926 (0.5857)


# Pull the TPOT models as basic models, attach QRF and do a stacking prediction

In [32]:
pipe_a = exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy,validate=False),
        MinMaxScaler()
    ),
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=True, max_depth=5, max_features=0.15000000000000002, min_samples_leaf=0.055, min_samples_split=0.505, n_estimators=100)),
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=False, max_depth=9, max_features=0.15000000000000002, min_samples_leaf=0.255, min_samples_split=0.005, n_estimators=100)),
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=True, max_depth=5, max_features=0.35000000000000003, min_samples_leaf=0.20500000000000002, min_samples_split=0.35500000000000004, n_estimators=250)),
    LGBMRegressor(colsample_bytree=0.75, learning_rate=0.01, max_bin=127, max_depth=4, min_child_weight=15, n_estimators=300, num_leaves=90, objective="fair", reg_alpha=0.05, subsample=0.75, subsample_freq=0, verbosity=-1,n_thread=-1)
)

pipe_b = make_pipeline(
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=True, max_depth=4, max_features=0.5500000000000002, min_samples_leaf=0.055, min_samples_split=0.455, n_estimators=300)),
    MinMaxScaler(),
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=True, max_depth=5, max_features=0.5500000000000002, min_samples_leaf=0.255, min_samples_split=0.20500000000000002, n_estimators=500)),
    LGBMRegressor(colsample_bytree=1.0, learning_rate=0.2, max_bin=63, max_depth=4, min_child_weight=0.001, n_estimators=250, num_leaves=90, objective="huber", reg_alpha=0.007, subsample=0.8, subsample_freq=0, verbosity=-1,n_thread=-1)
)

pipe_c = exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy,validate=False),
        make_union(
            FunctionTransformer(copy,validate=False),
            FunctionTransformer(copy,validate=False)
        )
    ),
    StackingEstimator(estimator=LGBMRegressor(colsample_bytree=1.0, learning_rate=0.005, max_bin=127, max_depth=5, min_child_weight=1, n_estimators=250, num_leaves=100, objective="mape", reg_alpha=0.01, subsample=0.7, subsample_freq=10, verbosity=-1)),
    LGBMRegressor(colsample_bytree=1.0, learning_rate=0.001, max_bin=127, max_depth=4, min_child_weight=10, n_estimators=300, num_leaves=70, objective="mape", reg_alpha=0.05, subsample=0.9, subsample_freq=30, verbosity=-1,n_thread=-1)
)


stacked_averaged_models = StackingAveragedModels(base_models = (pipe_a, pipe_b,pipe_c,model_xgb),
                                                 meta_model = ENet)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))


Stacking Averaged models score: -3.3235 (0.3430)


In [34]:
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))


Stacking Averaged models score: -3.3235 (0.3430)


# Final predictions:



In [55]:
from scipy.stats import boxcox
from scipy.special import inv_boxcox
preds_all = []

for year in [2016,2017]:
    print(year)
    data_dict = pd.read_pickle(f'../data/processed/data_dict_all.pkl')
    X_test = data_dict[year]['X_test_ts'].copy()
    
    tgt = "rougher.output.recovery"
    X = data_dict[year]['X_train_ts'].copy()
    print(f'1) Test shape: {X_test.shape}, train: {X.shape}')
    y = data_dict[year]['y_train'][tgt].dropna()
    y = y[(y>5) & (y <100)]
    
    inds = X.index.intersection(y.index)
    X = X.loc[inds]
    y = y.loc[inds]
        
    stacked_averaged_models.fit(X.values, y)
    ypred_r = stacked_averaged_models.predict(X_test.values)
    preds_r = pd.DataFrame(data = {'date':X_test.index, tgt:ypred_r}).set_index('date')
    
    tgt = "final.output.recovery"
    print(f'2) Test shape: {X_test.shape}, train: {X.shape}')
    
    X = data_dict[year]['X_train_ts'].copy()
    X_test = data_dict[year]['X_test_ts'].copy()
    y = data_dict[year]['y_train'][tgt].dropna()
    
    
    
    y = y[(y>5) & (y <100)]
    inds = X.index.intersection(y.index)
    X = X.loc[inds]
    y = y.loc[inds]
    print(f'3) Test shape: {X_test.shape}, train: {X.shape}')
    
    stacked_averaged_models.fit(X.values, y)
    ypred_f = stacked_averaged_models.predict(X_test.values)
    preds_f = pd.DataFrame(data = {'date':X_test.index, tgt:ypred_f}).set_index('date')

    preds_all.append(preds_r.join(preds_f))

2016
1) Test shape: (2928, 153), train: (16859, 153)
2) Test shape: (2928, 153), train: (5541, 153)
3) Test shape: (2928, 153), train: (5741, 153)
2017
1) Test shape: (2928, 153), train: (16859, 153)
2) Test shape: (2928, 153), train: (5128, 153)
3) Test shape: (2928, 153), train: (5613, 153)


In [56]:
stacked_preds_sub = pd.concat(preds_all)
stacked_preds_sub = stacked_preds_sub.reset_index()
stacked_preds_sub['date'] = stacked_preds_sub['date'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
stacked_preds_sub.set_index('date',inplace=True)
stacked_preds_sub.drop_duplicates(inplace=True)
stacked_preds_sub.to_csv('../results/stacked_sub_3_window.csv')

(8784, 153)

In [124]:
ypred_r

array([89.82622333, 89.75616916, 89.5315029 , ..., 81.3202456 ,
       80.8921535 , 81.34507872])